# 第7章: チャットアプリケーションの構築
## Azure OpenAI API クイックスタート


## 概要
このノートブックは、[Azure OpenAI Samples Repository](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst) から適応されたもので、[OpenAI](notebook-openai.ipynb) サービスにもアクセスするノートブックが含まれています。

Python OpenAI API は、いくつかの修正を加えることで Azure OpenAI とも連携します。違いの詳細については、こちらをご覧ください: [Python で OpenAI と Azure OpenAI エンドポイントを切り替える方法](https://learn.microsoft.com/azure/ai-services/openai/how-to/switching-endpoints?WT.mc_id=academic-109527-jasmineg)

さらに多くのクイックスタート例については、公式の [Azure OpenAI クイックスタート ドキュメント](https://learn.microsoft.com/azure/cognitive-services/openai/quickstart?pivots=programming-language-studio&WT.mc_id=academic-105485-koreyst) を参照してください。


## 目次  

[概要](../../../../07-building-chat-applications/python)  
[Azure OpenAI サービスの開始方法](../../../../07-building-chat-applications/python)  
[最初のプロンプトを作成する](../../../../07-building-chat-applications/python)  

[ユースケース](../../../../07-building-chat-applications/python)    
[1. テキストの要約](../../../../07-building-chat-applications/python)  
[2. テキストの分類](../../../../07-building-chat-applications/python)  
[3. 新しい製品名の生成](../../../../07-building-chat-applications/python)  
[4. 分類器のファインチューニング](../../../../07-building-chat-applications/python)  
[5. 埋め込み](../../../../07-building-chat-applications/python)

[参考文献](../../../../07-building-chat-applications/python)


### Azure OpenAI サービスの始め方

新規のお客様は、Azure OpenAI サービスへの[アクセス申請](https://aka.ms/oai/access?WT.mc_id=academic-105485-koreyst)が必要です。  
承認が完了した後、お客様は Azure ポータルにログインし、Azure OpenAI サービスのリソースを作成して、スタジオを通じてモデルの実験を開始できます。  

[すぐに始めるための優れたリソース](https://techcommunity.microsoft.com/blog/educatordeveloperblog/azure-openai-service-is-now-generally-available/3719177?WT.mc_id=academic-105485-koreyst)


### 最初のプロンプトを作成する  
この短い演習では、シンプルなタスク「要約」のためにOpenAIモデルにプロンプトを送信する基本的な方法を紹介します。


**手順**:  
1. Python環境にOpenAIライブラリをインストールする  
2. 標準のヘルパーライブラリを読み込み、作成したOpenAIサービスの典型的なOpenAIセキュリティ資格情報を設定する  
3. タスクに適したモデルを選択する  
4. モデル用のシンプルなプロンプトを作成する  
5. モデルAPIにリクエストを送信する！


### 1. OpenAIをインストールする


  > [!注意] このステップは、Codespaces または Devcontainer 内でこのノートブックを実行する場合は必要ありません


In [ ]:
%pip install openai python-dotenv

### 2. ヘルパーライブラリのインポートと認証情報のインスタンス化


In [ ]:
import os
from openai import AzureOpenAI
import numpy as np
from dotenv import load_dotenv
load_dotenv()

#validate data inside .env file

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

### 3. 適切なモデルの選択  
GPT-3.5-turbo または GPT-4 モデルは自然言語の理解と生成が可能です。このサービスでは、異なるタスクに適した異なるレベルの性能と速度を持つ4つのモデル機能を提供しています。

[Azure OpenAI models](https://learn.microsoft.com/azure/cognitive-services/openai/concepts/models?WT.mc_id=academic-105485-koreyst)


In [ ]:
# Select the General Purpose curie model for text
model = os.environ['AZURE_OPENAI_DEPLOYMENT']

## 4. プロンプト設計  

「大規模言語モデルの魔法は、膨大な量のテキストに対してこの予測誤差を最小化するように訓練されることで、これらの予測に役立つ概念を学習することにあります。例えば、彼らは次のような概念を学びます」(1):

* つづりの仕方
* 文法の仕組み
* 言い換えの方法
* 質問への回答方法
* 会話の進め方
* 多言語での執筆方法
* コーディング方法
* その他

#### 大規模言語モデルの制御方法  
「大規模言語モデルへのすべての入力の中で、圧倒的に最も影響力があるのはテキストプロンプトです」(1)。

大規模言語モデルは、いくつかの方法で出力を促すことができます:

- 指示: モデルに何を望むか伝える
- 補完: 欲しいものの始まりをモデルに完成させさせる
- デモンストレーション: モデルに何を望むか示す、以下のいずれかで:
- プロンプト内のいくつかの例
- 微調整のトレーニングデータセット内の何百または何千もの例



#### プロンプト作成の基本的な3つのガイドライン:

**見せて伝える**。指示、例、またはその両方の組み合わせを通じて、何を望んでいるかを明確に示してください。モデルにアイテムのリストをアルファベット順に並べ替えさせたい場合や、段落を感情で分類させたい場合は、それが望みであることを示してください。

**質の高いデータを提供する**。分類器を構築しようとしている場合やモデルにパターンを従わせたい場合は、十分な例があることを確認してください。例を校正することを忘れないでください — モデルは通常、基本的なつづりの間違いを見抜いて応答を返すほど賢いですが、それが意図的だと判断し、応答に影響を与えることもあります。

**設定を確認する**。temperature と top_p の設定は、モデルが応答を生成する際の決定論的な度合いを制御します。正解が一つだけの応答を求める場合は、これらを低く設定したいでしょう。より多様な応答を求める場合は、高く設定することがあります。これらの設定で人々が最もよく犯す間違いは、これらを「賢さ」や「創造性」のコントロールだと誤解することです。


出典: https://learn.microsoft.com/azure/ai-services/openai/overview


画像はあなたの最初のテキストプロンプトを作成しています！


### 5. 提出！


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

### 同じ呼び出しを繰り返した場合、結果はどう比較されますか？


In [ ]:

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## テキストの要約  
#### 課題  
テキストの最後に「tl;dr:」を追加して要約します。モデルが追加の指示なしで多くのタスクをどのように理解して実行するかに注目してください。tl;drよりも詳細なプロンプトを使ってモデルの動作を変更し、受け取る要約をカスタマイズすることもできます(3)。  

最近の研究では、大規模なテキストコーパスで事前学習を行い、その後特定のタスクでファインチューニングすることで、多くのNLPタスクやベンチマークで大幅な成果が示されています。通常はタスクに依存しないアーキテクチャですが、この方法でも数千から数万の例を含むタスク固有のファインチューニングデータセットが必要です。これに対し、人間は一般的に数例や簡単な指示だけで新しい言語タスクを実行できますが、現在のNLPシステムはこれがまだ大きな課題となっています。ここでは、言語モデルの規模を拡大することで、タスクに依存しない少数ショットの性能が大幅に向上し、時には従来の最先端ファインチューニング手法と競合するレベルに達することを示します。  

Tl;dr


# いくつかのユースケースの演習  
1. テキストの要約  
2. テキストの分類  
3. 新しい製品名の生成  
4. 埋め込み  
5. 分類器のファインチューニング


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\ntl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## テキスト分類  
#### チャレンジ  
推論時に提供されたカテゴリにアイテムを分類します。以下の例では、プロンプト(*playground_reference)でカテゴリと分類するテキストの両方を提供しています。

顧客からの問い合わせ: こんにちは、最近ノートパソコンのキーボードのキーの一つが壊れてしまい、交換が必要です。

分類されたカテゴリ:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## 新しい製品名を生成する
#### チャレンジ
例の単語から製品名を作成します。ここでは、生成する製品に関する情報をプロンプトに含めています。また、希望するパターンを示すために類似の例も提供しています。ランダム性とより革新的な応答を増やすために温度値も高く設定しています。

製品説明：家庭用ミルクセーキメーカー  
シードワード：速い、健康的、コンパクト。  
製品名：HomeShaker、Fit Shaker、QuickShake、Shake Maker

製品説明：どんな足のサイズにも合う靴のペア。  
シードワード：適応可能、フィット、オムニフィット。


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

## 埋め込み
このセクションでは、埋め込みの取得方法と、単語、文、ドキュメント間の類似性の見つけ方を示します。以下のノートブックを実行するには、`text-embedding-ada-002` をベースモデルとして使用するモデルをデプロイし、.env ファイル内の `AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT` 変数にそのデプロイメント名を設定する必要があります。


### モデル分類 - 埋め込みモデルの選択

**モデル分類**: {family} - {capability} - {input-type} - {identifier}  

{family}     --> text-embedding  (埋め込みファミリー)  
{capability} --> ada             (その他の埋め込みモデルは2024年に廃止予定)  
{input-type} --> n/a             (検索モデルのみ指定)  
{identifier} --> 002             (バージョン002)  

model = 'text-embedding-ada-002'


  > [!注意] この手順は、Codespaces または Devcontainer 内でこのノートブックを実行する場合は必要ありません


In [ ]:
# Dependencies for embeddings_utils
%pip install matplotlib plotly scikit-learn pandas

In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [ ]:
text = 'the quick brown fox jumped over the lazy dog'
model= os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']
client.embeddings.create(input='[text]', model=model).data[0].embedding

In [ ]:

# compare several words
automobile_embedding  = client.embeddings.create(input='automobile', model=model).data[0].embedding
vehicle_embedding     = client.embeddings.create(input='vehicle', model=model).data[0].embedding
dinosaur_embedding    = client.embeddings.create(input='dinosaur', model=model).data[0].embedding
stick_embedding       = client.embeddings.create(input='stick', model=model).data[0].embedding

print(cosine_similarity(automobile_embedding, vehicle_embedding))
print(cosine_similarity(automobile_embedding, dinosaur_embedding))
print(cosine_similarity(automobile_embedding, stick_embedding))

## cnn daily news データセットからの記事の比較
source: https://huggingface.co/datasets/cnn_dailymail


In [ ]:
import pandas as pd
cnn_daily_articles = ['BREMEN, Germany -- Carlos Alberto, who scored in FC Porto\'s Champions League final victory against Monaco in 2004, has joined Bundesliga club Werder Bremen for a club record fee of 7.8 million euros ($10.7 million). Carlos Alberto enjoyed success at FC Porto under Jose Mourinho. "I\'m here to win titles with Werder," the 22-year-old said after his first training session with his new club. "I like Bremen and would only have wanted to come here." Carlos Alberto started his career with Fluminense, and helped them to lift the Campeonato Carioca in 2002. In January 2004 he moved on to FC Porto, who were coached by José Mourinho, and the club won the Portuguese title as well as the Champions League. Early in 2005, he moved to Corinthians, where he impressed as they won the Brasileirão,but in 2006 Corinthians had a poor season and Carlos Alberto found himself at odds with manager, Emerson Leão. Their poor relationship came to a climax at a Copa Sul-Americana game against Club Atlético Lanús, and Carlos Alberto declared that he would not play for Corinthians again while Leão remained as manager. Since January this year he has been on loan with his first club Fluminense. Bundesliga champions VfB Stuttgart said on Sunday that they would sign a loan agreement with Real Zaragoza on Monday for Ewerthon, the third top Brazilian player to join the German league in three days. A VfB spokesman said Ewerthon, who played in the Bundesliga for Borussia Dortmund from 2001 to 2005, was expected to join the club for their pre-season training in Austria on Monday. On Friday, Ailton returned to Germany where he was the league\'s top scorer in 2004, signing a one-year deal with Duisburg on a transfer from Red Star Belgrade. E-mail to a friend .',
                        '(CNN) -- Football superstar, celebrity, fashion icon, multimillion-dollar heartthrob. Now, David Beckham is headed for the Hollywood Hills as he takes his game to U.S. Major League Soccer. CNN looks at how Bekham fulfilled his dream of playing for Manchester United, and his time playing for England. The world\'s famous footballer has begun a five-year contract with the Los Angeles Galaxy team, and on Friday Beckham will meet the press and reveal his new shirt number. This week, we take an in depth look at the life and times of Beckham, as CNN\'s very own "Becks," Becky Anderson, sets out to examine what makes the man tick -- as footballer, fashion icon and global phenomenon. It\'s a long way from the streets of east London to the Hollywood Hills and Becky charts Beckham\'s incredible rise to football stardom, a journey that has seen his skills grace the greatest stages in world soccer. She goes in pursuit of the current hottest property on the sports/celebrity circuit in the U.S. and along the way explores exactly what\'s behind the man with the golden boot. CNN will look back at the life of Beckham, the wonderfully talented youngster who fulfilled his dream of playing for Manchester United, his marriage to pop star Victoria, and the trials and tribulations of playing for England. We\'ll look at the highs (scoring against Greece), the lows (being sent off during the World Cup), the Man. U departure for the Galacticos of Madrid -- and now the Home Depot stadium in L.A. We\'ll ask how Beckham and his family will adapt to life in Los Angeles -- the people, the places to see and be seen and the celebrity endorsement. Beckham is no stranger to exposure. He has teamed with Reggie Bush in an Adidas commercial, is the face of Motorola, is the face on a PlayStation game and doesn\'t need fashion tips as he has his own international clothing line. But what does the star couple need to do to become an accepted part of Tinseltown\'s glitterati? The road to major league football in the U.S.A. is a well-worn route for some of the world\'s greatest players. We talk to some of the former greats who came before him and examine what impact these overseas stars had on U.S. soccer and look at what is different now. We also get a rare glimpse inside the David Beckham academy in L.A, find out what drives the kids and who are their heroes. The perception that in the U.S.A. soccer is a "game for girls" after the teenage years is changing. More and more young kids are choosing the European game over the traditional U.S. sports. E-mail to a friend .',
                        'LOS ANGELES, California (CNN) -- Youssif, the 5-year-old burned Iraqi boy, rounded the corner at Universal Studios when suddenly the little boy hero met his favorite superhero. Youssif has always been a huge Spider-Man fan. Meeting him was "my favorite thing," he said. Spider-Man was right smack dab in front of him, riding a four-wheeler amid a convoy of other superheroes. The legendary climber of buildings and fighter of evil dismounted, walked over to Youssif and introduced himself. Spidey then gave the boy from a far-away land a gentle hug, embracing him in his iconic blue and red tights. He showed Youssif a few tricks, like how to shoot a web from his wrist. Only this time, no web was spun. "All right Youssif!" Spider-Man said after the boy mimicked his wrist movement. Other superheroes crowded around to get a closer look. Even the Green Goblin stopped his villainous ways to tell the boy hi. Youssif remained unfazed. He didn\'t take a liking to Spider-Man\'s nemesis. Spidey was just too cool. "It was my favorite thing," the boy said later. "I want to see him again." He then felt compelled to add: "I know it\'s not the real Spider-Man." This was the day of dreams when the boy\'s nightmares were, at least temporarily, forgotten. He met SpongeBob, Lassie and a 3-year-old orangutan named Archie. The hairy, brownish-red primate took to the boy, grabbing his hand and holding it. Even when Youssif pulled away, Archie would inch his hand back toward the boy\'s and then snatch it. See Youssif enjoy being a boy again » . The boy giggled inside a play area where sponge-like balls shot out of toy guns. It was a far different artillery than what he was used to seeing in central Baghdad, as recently as a week ago. He squealed with delight and raced around the room collecting as many balls as he could. He rode a tram through the back stages at Universal Studios. At one point, the car shook. Fire and smoke filled the air, debris cascaded down and a big rig skidded toward the vehicle. The boy and his family survived the pretend earthquake unscathed. "Even I was scared," the dad said. "Well, I wasn\'t," Youssif replied. The father and mother grinned from ear to ear throughout the day. Youssif pushed his 14-month-old sister, Ayaa, in a stroller. "Did you even need to ask us if we were interested in coming here?" Youssif\'s father said in amazement. "Other than my wedding day, this is the happiest day of my life," he said. Just a day earlier, the mother and father talked about their journey out of Iraq and to the United States. They also discussed that day nine months ago when masked men grabbed their son outside the family home, doused him in gas and set him on fire. His mother heard her boy screaming from inside. The father sought help for his boy across Baghdad, but no one listened. He remembers his son\'s two months of hospitalization. The doctors didn\'t use anesthetics. He could hear his boy\'s piercing screams from the other side of the hospital. Watch Youssif meet his doctor and play with his little sister » . The father knew that speaking to CNN would put his family\'s lives in jeopardy. The possibility of being killed was better than seeing his son suffer, he said. "Anything for Youssif," he said. "We had to do it." They described a life of utter chaos in Baghdad. Neighbors had recently given birth to a baby girl. Shortly afterward, the father was kidnapped and killed. Then, there was the time when some girls wore tanktops and jeans. They were snatched off the street by gunmen. The stories can be even more gruesome. The couple said they had heard reports that a young girl was kidnapped and beheaded --and her killers sewed a dog\'s head on the corpse and delivered it to her family\'s doorstep. "These are just some of the stories," said Youssif\'s mother, Zainab. Under Saddam Hussein, there was more security and stability, they said. There was running water and electricity most of the time. But still life was tough under the dictator, like the time when Zainab\'s uncle disappeared and was never heard from again after he read a "religious book," she said. Sitting in the parking lot of a Target in suburban Los Angeles, Youssif\'s father watched as husbands and wives, boyfriends and girlfriends, parents and their children, came and went. Some held hands. Others smiled and laughed. "Iraq finished," he said in what few English words he knows. He elaborated in Arabic: His homeland won\'t be enjoying such freedoms anytime soon. It\'s just not possible. Too much violence. Too many killings. His two children have only seen war. But this week, the family has seen a much different side of America -- an outpouring of generosity and a peaceful nation at home. "It\'s been a dream," the father said. He used to do a lot of volunteer work back in Baghdad. "Maybe that\'s why I\'m being helped now," the father said. At Universal Studios, he looked out across the valley below. The sun glistened off treetops and buildings. It was a picturesque sight fit for a Hollywood movie. "Good America, good America," he said in English. E-mail to a friend . CNN\'s Arwa Damon contributed to this report.'
]

cnn_daily_article_highlights = ['Werder Bremen pay a club record $10.7 million for Carlos Alberto .\nThe Brazilian midfielder won the Champions League with FC Porto in 2004 .\nSince January he has been on loan with his first club, Fluminense .',
                                'Beckham has agreed to a five-year contract with Los Angeles Galaxy .\nNew contract took effect July 1, 2007 .\nFormer English captain to meet press, unveil new shirt number Friday .\nCNN to look at Beckham as footballer, fashion icon and global phenomenon .',
                                'Boy on meeting Spider-Man: "It was my favorite thing"\nYoussif also met SpongeBob, Lassie and an orangutan at Universal Studios .\nDad: "Other than my wedding day, this is the happiest day of my life"'
]

cnn_df = pd.DataFrame({"articles":cnn_daily_articles, "highligths":cnn_daily_article_highlights})

cnn_df.head()

In [ ]:
article1_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[0], model=model).data[0].embedding
article2_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[1], model=model).data[0].embedding
article3_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[2], model=model).data[0].embedding

print(cosine_similarity(article1_embedding, article2_embedding))
print(cosine_similarity(article1_embedding, article3_embedding))

# References  
- [Azure Documentation - Azure OpenAI Models](https://learn.microsoft.com/azure/cognitive-services/openai/concepts/models?WT.mc_id=academic-105485-koreyst)  
- [OpenAI Studio Examples](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  


# さらなる支援について  
[OpenAI Commercialization Team](AzureOpenAITeam@microsoft.com)


# 貢献者
* Louis Li  


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**免責事項**：  
本書類はAI翻訳サービス「Co-op Translator」（https://github.com/Azure/co-op-translator）を使用して翻訳されました。正確性の向上に努めておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があります。原文の言語によるオリジナル文書が正式な情報源とみなされるべきです。重要な情報については、専門の人間による翻訳を推奨します。本翻訳の利用により生じた誤解や誤訳について、当方は一切の責任を負いかねます。
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
